In [1]:
pip install scx>=1.0.4

Note: you may need to restart the kernel to use updated packages.


In [2]:
from scx.optimize import Model

In [3]:
transport = [
    {
        'origin_name':'A1',
        'destination_name':'R1',
        'distance': 105,
        'cost_per_mile':0.12,
    },
    {
        'origin_name':'A1',
        'destination_name':'R2',
        'distance': 256,
        'cost_per_mile':0.12,
    },
    {
        'origin_name':'A1',
        'destination_name':'R3',
        'distance': 86,
        'cost_per_mile':0.12,
    },
    {
        'origin_name':'A2',
        'destination_name':'R1',
        'distance': 240,
        'cost_per_mile':0.12,
    },
    {
        'origin_name':'A2',
        'destination_name':'R2',
        'distance': 136,
        'cost_per_mile':0.12,
    },
    {
        'origin_name':'A2',
        'destination_name':'R3',
        'distance': 198,
        'cost_per_mile':0.12,
    },
]

demand = [
    {
        'name':'R1', 
        'demand':2500
    },
    {
        'name':'R2', 
        'demand':4350
    },
    {
        'name':'R3', 
        'demand':3296
    },
]

In [4]:
for t in transport:
    # Create decision variables for each item in transport
    t['amt']=Model.variable(name=f"{t['origin_name']}__{t['destination_name']}__amt", lowBound=0)
    # Calculate the variable cost of shipping for each item in tranport
    t['cost']=t['distance']*t['cost_per_mile']
    # Calculate the variable profit per item shipped
    t['profit']=45-t['cost']

In [5]:
# Initialize the my_model
my_model = Model(name="Blinky22", sense='maximize')


# Add the Objective Fn
my_model.add_objective(
    fn=Model.sum([t['amt']*t['profit'] for t in transport])
)

# Add Constraints
## Demand Constraint
for d in demand:
    my_model.add_constraint(
        name=f"{d['name']}__demand",
        fn=my_model.sum([t['amt'] for t in transport if t['destination_name']==d['name']]) <= d['demand']
    )

# Solve the my_model
my_model.solve()

In [6]:
my_model.show_formulation()

Blinky22:
MAXIMIZE
32.4*A1__R1__amt + 14.280000000000001*A1__R2__amt + 34.68*A1__R3__amt + 16.200000000000003*A2__R1__amt + 28.68*A2__R2__amt + 21.240000000000002*A2__R3__amt + 0.0
SUBJECT TO
R1__demand: A1__R1__amt + A2__R1__amt <= 2500

R2__demand: A1__R2__amt + A2__R2__amt <= 4350

R3__demand: A1__R3__amt + A2__R3__amt <= 3296

VARIABLES
A1__R1__amt Continuous
A1__R2__amt Continuous
A1__R3__amt Continuous
A2__R1__amt Continuous
A2__R2__amt Continuous
A2__R3__amt Continuous



In [7]:
# Show the outputs
my_model.show_outputs()

{'objective': 320063.28,
 'status': 'Optimal',
 'variables': {'A1__R1__amt': 2500.0,
               'A1__R2__amt': 0.0,
               'A1__R3__amt': 3296.0,
               'A2__R1__amt': 0.0,
               'A2__R2__amt': 4350.0,
               'A2__R3__amt': 0.0}}
